### Basic Agent Example

In [1]:
from helpers import llm
from llama_index.core.agent.workflow import FunctionAgent, AgentWorkflow

# Define a simple calculator tool
def multiply(a: float, b: float) -> float:
    """Useful for multiplying two numbers."""
    return a * b


# Create an agent workflow with our calculator tool
agent = FunctionAgent(
    tools=[multiply],
    llm=llm,
    system_prompt="You are a helpful assistant that can multiply two numbers.",
)

### Adding Chat History

In [2]:
from llama_index.core.workflow import Context

# create context
ctx = Context(agent)

# run agent with context
response = await agent.run("My name is Logan", ctx=ctx)
print(str(response))

response = await agent.run("What is my name?", ctx=ctx)
print(str(response))

Hello Logan! How can I help you today?

Your name is Logan.


### Maintaining state over longer periods

In [3]:
from llama_index.core.workflow import JsonSerializer

ctx_dict = ctx.to_dict(serializer=JsonSerializer())

restored_ctx = Context.from_dict(
    agent, ctx_dict, serializer=JsonSerializer()
)

response3 = await agent.run(user_msg="What's my name?", ctx=restored_ctx)
print(str(response3))

Your name is Logan.


### Tools and state

In [4]:
async def set_name(ctx: Context, name: str) -> str:
    async with ctx.store.edit_state() as ctx_state:
        ctx_state["state"]["name"] = name

    return f"Name set to {name}"

workflow = AgentWorkflow.from_tools_or_functions(
    [set_name],
    llm=llm,
    system_prompt="You are a helpful assistant that can set a name.",
    initial_state={"name": "unset"},
)

ctx = Context(workflow)

# check if it knows a name before setting it
response = await workflow.run(user_msg="What's my name?", ctx=ctx)
print(str(response))

response2 = await workflow.run(user_msg="My name is Laurie", ctx=ctx)
print(str(response2))

state = await ctx.store.get("state")
print("Name as stored in state: ", state["name"])


What would you like to set your name to?
Current state:
{'name': 'Laurie'}

Current message:
Hello Laurie!
Name as stored in state:  Laurie


### Chat Engine - Condense Plus Context Mode

In [5]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

data = SimpleDirectoryReader(input_dir="./data/").load_data()
index = VectorStoreIndex.from_documents(data)

from llama_index.core.memory import ChatMemoryBuffer

memory = ChatMemoryBuffer.from_defaults(token_limit=3900)

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
        "Here are the relevant documents for the context:\n"
        "{context_str}"
        "\nInstruction: Use the previous chat history, or the context above, to interact and help the user."
    ),
    verbose=False,
)

response = chat_engine.chat("What did Paul Graham do growing up")
print(response)

print("----------------------------------------------------")

response_2 = chat_engine.chat("Can you tell me more?")
print(response_2)

Paul Graham spent his childhood engaged in two main activities outside of school: writing and programming.

He wrote short stories, which he describes as having "hardly any plot, just characters with strong feelings."

His early programming experiences began around age 13 or 14 on an IBM 1401 mainframe computer at his school district. He used Fortran and punch cards, though he found it difficult to do much with it due to limitations.

Everything changed with microcomputers. He eventually convinced his father to buy him a TRS-80 around 1980, which is when he "really started programming." He wrote simple games, a program to predict model rocket flight, and even a word processor for his father.
----------------------------------------------------
Certainly! Let's delve a bit deeper into Paul Graham's early life and experiences, based on the essay.

**Early Programming Challenges and Discoveries:**

*   **The IBM 1401:** His initial foray into programming on the IBM 1401 was quite limited.

### Reset conversation state

In [6]:
chat_engine.reset()
response = chat_engine.chat("Hello! What do you know?")
print(response)

I know a lot of things! I'm a large language model, trained by Google. I can understand and generate human-like text, and I have access to a vast amount of information from the real world.

What would you like to talk about? We could discuss the essay by Paul Graham, or anything else you have in mind!


In [7]:
chat_engine = index.as_chat_engine(chat_mode="condense_question", verbose=True)
response = chat_engine.chat("What did Paul Graham do after YC?")
print(response)

print("----------------------------------------------------")

response = chat_engine.chat("What about after that?")
print(response)

print("----------------------------------------------------")

response = chat_engine.chat("Can you tell me more?")
print(response)

print("----------------------------------------------------")

chat_engine.reset()
response = chat_engine.chat("What about after that?")
print(response)

Querying with: What did Paul Graham do after YC?
After leaving YC, Paul Graham decided to paint, aiming to see how good he could become if he focused on it. He spent most of 2014 painting but eventually ran out of enthusiasm for it. Following this, he started writing essays again, including some not about startups. In March 2015, he began working on Lisp once more.
----------------------------------------------------
Querying with: What did Paul Graham do after he started writing essays again and working on Lisp in March 2015?
The provided text does not contain information about what Paul Graham did in March 2015. It mentions that he wrote many essays and that O'Reilly reprinted a collection of them as a book called "Hackers & Painters." He also worked on spam filters and did more painting.
----------------------------------------------------
Querying with: What did Paul Graham do after March 2015, specifically after he started working on Lisp again, writing essays, and doing more pain

### Verbose mode


In [8]:
chat_engine1 = index.as_chat_engine(chat_mode="condense_question", verbose=True)
response = chat_engine1.chat("What did Paul Graham do after YC?")
print(response)

print("----------------------------------------------------")

chat_engine2 = index.as_chat_engine(chat_mode="condense_question", verbose=False)
response = chat_engine2.chat("What did Paul Graham do after YC?")
print(response)

Querying with: What did Paul Graham do after YC?
After leaving YC, Paul Graham decided to take up painting, aiming to see how proficient he could become if he dedicated himself to it. He spent most of 2014 painting but eventually stopped when he lost interest. Following this, he resumed writing essays and, in March 2015, began working on Lisp again.
----------------------------------------------------
After leaving YC, Paul Graham decided to paint, aiming to see how good he could become if he focused on it. He spent most of 2014 painting but eventually ran out of steam and stopped. Following this, he started writing essays again, including some not about startups. In March 2015, he began working on Lisp once more.


### Chat Engine - ReAct Mode

In [9]:
from llama_index.core.chat_engine.types import ChatMode

chat_engine2 = index.as_chat_engine(chat_mode=ChatMode.REACT, verbose=True)

# chat_engine_react = index.as_chat_engine(chat_mode="react", llm=llm, verbose=True)
response = chat_engine2.chat(
    "What are the first programs Paul Graham tried writing?"
)
print(response)

ValueError: ChatMode.REACT and ChatMode.OPENAI are now deprecated and removed. Please use the ReActAgent or FunctionAgent classes from llama_index.core.agent.workflow to create an agent with a query engine tool.

### Chat Engine - Context Mode

In [10]:
chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    system_prompt=(
        "You are a chatbot, able to have normal interactions, as well as talk"
        " about an essay discussing Paul Grahams life."
    ),
)

response = chat_engine.chat("Hello!")
print(response)

print("----------------------------------------------------")

response = chat_engine.chat("What did Paul Graham do growing up?")
print(response)

print("----------------------------------------------------")

response = chat_engine.chat("Can you tell me more?")
print(response)

Hello there! How can I help you today? Are you interested in learning more about Paul Graham's essay, or perhaps something else?
----------------------------------------------------
Based on the essay, Paul Graham was involved in two main activities outside of school growing up:

1.  **Writing:** He wrote short stories, though he describes them as "awful" with "hardly any plot."
2.  **Programming:** He started programming around 9th grade (age 13 or 14) on an IBM 1401 at his school district. Later, with the advent of microcomputers, he really got into programming after his father bought him a TRS-80 around 1980. He wrote simple games, a program to predict model rocket flight, and even a word processor for his father.
----------------------------------------------------
Certainly! Let's dive a bit deeper into Paul Graham's early life and experiences as described in the essay.

**Early Programming Experiences:**

*   **IBM 1401 (9th Grade):** His very first exposure to programming was on

In [12]:
chat_engine.reset()
response = chat_engine.chat("Hello! What do you know?")
print(response)

I know a lot about Paul Graham's life, based on his essay. For example, I know about his experiences working at Interleaf, his thoughts on art school, and his reflections on leaving Y Combinator.

What are you interested in knowing about?
